<h1><center>  <font color='blue'>Team Building on a Quantum Computer</font> </center></h1>

![divimg.png](divimg.png)

<br/> Today, we're going to use a Quantum Computer to sort ourselves into roles within a team!

We've learned so far that Quantum Computers can solve a number of problems in various industries. We also learned that there is a particular type of quantum computing called $\textbf{Quantum Annealing}$, which is suited to solving $\it{optimization}$ problems.

<img src="maxresdefault.jpg" width="200" style="float:right">

$\it{Optimization}$ is the process of finding the most efficient use of time or resources. If you were delivering mail to several houses in several neighbourhoods, you may want to optimize your route such that you find a route that will  <br/> allow you to deliver mail in the least amount of time. This 'optimal' route would also use the least amount of gas (if you're driving), and least amount of energy (if you're walking or biking). Often, you'll be maximizing or minimizing something (i.e. minimizing time, or maximizing how much mail you deliver a day)



### Let's dive into an optimization problem

Suppose that you're in a team of 5, working on a new product. In team settings, communication is key. If all 5 of you try working on your own, you may find that work is being done twice, and progress is slow.

You may find it easier to split yourselves among roles. However, if you just assign roles randomly to each group member, people may end up working on roles that aren't their preferred roles. Two team members may prefer each other's roles, in which case this process is quite inefficient and can lead to conflict or less enthusiastic team members.

You decide to take on the first role: Project Manager. You've been voted into your position unanimously. It is now your responsibility to assign roles to each team member.

### How would you go about this?

Write your strategy into the next cell and hit Shift and Enter Key (at the same time) to lock in your answers
***

### ==================================================





### ==================================================

***


Perhaps the first thing you'd do is ask team members to let you know what role they'd like to take on. However, this can be particularly problematic if you have several people with the same first choice. You don't really have enough information to assign roles well here.

So you ask for their top 3 choices. This may work alright if there are only three roles, but you have 5 team members and five different roles.

Okay, so maybe the best thing to do is to get everyone to rank all the roles and all their choices. This'll be good (for the most part). However, you still don't know how strongly someone may feel about particular roles. For instance, you may have team members who are okay with all five roles (or some subset of roles), while other members are extremely passionate about only one role. In situations like these, you may feel that those who feel very strongly about one role be more likely to get it than someone who is equally happy with any role.

You decide that the best thing to do is for people to rank their preferences for all five roles, but to assign values from 1-3 instead of arbitrary rankings. 1 being that the role is their preferred choice, and scaling down to three as their least preferred choice.

***
### Quantum Computer

This is not an easy problem to solve by hand. How can you figure out what the best roles are based on numbers? It's a bit tricky, and gets even more challenging when there are more team members and more roles.

Luckily for you, this is a problem you can solve on a Quantum Computer. We will need to frame this as an optimization problem. In this case, we want to find the ideal roles such that if people don't get their first choice, most people get their second or third choices. We want to be as efficient as possible with our team, and so the hope is that everyone is happy with their positions.

Note that this is a relative assignment. In other words, it may be possible that not everyone is very happy with their role. For instance, if everyone submits the exact same rankings from 1-5, then the problem is making the most out of a challenging situation.
***

In [1]:
# Importing the necessary packages from D-Wave

import dimod
import itertools
from dwave.system import DWaveSampler, EmbeddingComposite

## The Design Challenge
<img src="group.png" width="200" style="float:right">

Get into groups of 4-5. You are all now working in the aerospace industry. Your mission? To fund, construct, and sell a shuttle that will land on the surface of Mars.

There are a few things to note here.
<ol/>
    <li/> You have a fixed budget of $100.00 </li>
    <li/>You will have to build a prototype of your shuttle. </li>
    <li/>You will be asked to conduct a 30-second pitch your prototype to potential investors. </li>
</ol>

There are also several $\textbf{technical requirements}$ for your design.

<ol/>
    <li/>Your astronaut must land safely, i.e. they should not fall out of the shuttle.</li>
    <li/>You are not allowed to cover the top of the shuttle, i.e. don't cover the top of the dixie cup.</li>
    <li/>You cannot place/tape/tie anything inside the shuttle. The only thing in the dixie cup must be the astronaut.</li>
</ol>

### Roles

You'll be using a quantum computer to sort your group into roles. Have each person in your group change one of the letters to their name.

In [2]:
people = ['A', 'B', 'C', 'D', 'E'] # People's names
roles = ['PM', 'PE', 'ME', 'FA', 'MP'] # All the roles
gamma = 1

These are the following roles.

<ol/>
    <li/>$\textbf{Project Manager (PM)}$</li>
    <li/>$\textbf{Physics Engineer (PE)}$</li>
    <li/>$\textbf{Mechanical Engineer (ME)}$</li>
    <li/>$\textbf{Financial Analyst (FA)}$</li>
    <li/>$\textbf{Marketing and Promotional (MP)}$</li>
</ol>

While you may have been assigned to a particular role, that does not mean that you cannot participate in the other roles. Rather, these roles are meant to assign leadership and responsibility to each aspect of the project.

Each team member should rank their preferences for each of the roles from 1 to 3, 1 being most preferred, and 3 being least preferred. Remember to change each letter (A to E) to the team member's name. Go into the following code and make the necessary changes. Hit Shift-Enter to run the code.

If you have less than 5 people in your team, feel free to remove a row of letters. Make sure that each line ends with a comma ',', except for the last line which ends in a '}' bracket.

In [3]:
people_roles = {'A PM': 2, 'A PE': 3, 'A ME': 2, 'A FA': 1, 'A MP': 2,
                'B PM': 2, 'B PE': 3, 'B ME': 3,'B FA': 2, 'B MP': 1,
                'C PM': 1, 'C PE': 3, 'C ME': 3, 'C FA': 2, 'C MP': 1,
                'D PM': 3, 'D PE': 2, 'D ME': 2, 'D FA': 1, 'D MP': 3,
                'E PM': 1, 'E PE': 1, 'E ME': 3, 'E FA': 1, 'E MP': 3}

Now we'll run the following code to turn our data and our problem into a format that can be embedded and run on a Quantum Computer.

In [4]:
# change the linear biases
for x in people_roles.keys():
    people_roles[x] = (people_roles[x] - 2)*gamma

bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)
bqm.linear = people_roles
bqm.offset = 5              # this helps us choose 5 people from the set



# Evaluate every potential edge in the graph
for i, j in itertools.combinations(range(len(people_roles)), 2):
    k = list(people_roles.keys())

    [person1, role1] = k[i].split()
    [person2, role2] = k[j].split()

    if person1 == person2 or role1 == role2:
        bqm.add_interaction(k[i], k[j], 4, dimod.BINARY)    # enforce strong pos quad bias if nodes have same person or role
    else:
        bqm.add_interaction(k[i], k[j], -1, dimod.BINARY)

# print(bqm)

sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(bqm, chain_strength=8, num_reads=100)

# print('\n')
#print(response)

# Print the variables chosen in the lowest energy sample
sample = response.samples()[0]
result = ''

for x in range(1):
    result = ''
    sample = response.samples()[x]

    for i in sample.keys():
        if sample[i] == 1:
            result = result + i + ', '

    result += ' cbf = ' + str(response.record['chain_break_fraction'][x])

    print(result)


A MP, B ME, C PM, D PE, E FA,  cbf = 0.0


You may not feel like these roles match your preferences. Run the program again and you may see that you get a different result! This is normal. We want to find the most efficient route, and so we often have to run the code many times to find the optimal result. In the code above, we're running the program, or reading the values 100 times ("num_reads=100")

### STORE

<ol/>
    <li/> Plates - CAD $\textbf{20}$ a plate </li>
    <li/> Pipe Cleaners - CAD $\textbf{10}$ a pipe cleaner </li>
    <li/> Skewers CAD $\textbf{10}$ a skewer </li>
    <li/> Tape = CAD $\textbf{20}$ an arm's length </li>
    <li/> Coffee Filters - CAD $\textbf{60}$ a coffee filter </li>
    <li/> Straws - CAD $\textbf{10}$ a paper straw </li>
    <li/> String - CAD $\textbf{40}$ an arm's length </li>

</ol>